In [1]:
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
from mpl_plot import *

import numpy as np

In [9]:
def load_mem_size(filename):
    a, p, g, m, v = [], [], [], [], []
    with open(filename, "rb") as f:
        for line in f.readlines():
            if (b"allocating" in line) and (b"for m" in line):
                m.append(float(line.split()[1]))
            if (b"allocating" in line) and (b"for v" in line):
                v.append(float(line.split()[1]))
            if (b"allocating" in line) and (b"for p" in line):
                p.append(float(line.split()[1]))
            if (b"allocating" in line) and (b"for g" in line):
                g.append(float(line.split()[1]))
            if (b"allocating" in line) and (b"for activations" in line):
                a.append(float(line.split()[1]))
    return a, p, g, m, v

result_f = "../log.stats.gpt"
a, p, g, m, v = load_mem_size(result_f)

In [15]:
n_gpt, n_batch = 5, 8
a = np.asarray(a).reshape((n_gpt, n_batch))
m = np.asarray(m).reshape((n_gpt, n_batch))
v = np.asarray(v).reshape((n_gpt, n_batch))
p = np.asarray(p).reshape((n_gpt, n_batch))
g = np.asarray(g).reshape((n_gpt, n_batch))
total = a + m + v + p + g

In [18]:
p_a = a / total
p_pg = (p + g) / total
p_mv = (m + v) / total

In [22]:
gpts = ["7B", "70B", "145B", "300B", "1T"]
batch_size = [1, 4, 8, 16, 32, 64, 128, 256]

colors = [crimson, blue, teal]

for i in range(5):
    gpt = gpts[i]
    
    x = list(range(1, len(batch_size)+1))
    y_a = p_a[i]
    y_pg = p_pg[i]
    y_mv = p_mv[i]
    
    fig_f = f"figures/{gpt}_memory.pdf"
    fig, axs = plt.subplots(figsize=(8, 6))
    
    axs.bar(x, y_a, color=colors[0], edgecolor=black)
    axs.bar(x, y_pg, bottom=y_a, color=colors[1], edgecolor=black)
    axs.bar(x, y_mv, bottom=y_a+y_pg, color=colors[2], edgecolor=black)
    axs.set_xticks(x, batch_size)
    x_label = "Batch Size"
    y_label = "Percentage %"
    axs.set_xlabel(x_label)
    axs.set_ylabel(y_label)
    fig.tight_layout()
    plt.savefig(fig_f)
    print(fig_f)

figures/7B_memory.pdf
figures/70B_memory.pdf
figures/145B_memory.pdf
figures/300B_memory.pdf
figures/1T_memory.pdf
